# Time Series Forecasting with GluonTS

This notebook demonstrates a simple time series forecasting approach using the GluonTS library with MXNet backend. The process includes data preparation, model training, prediction, and error computation. We will use a dataset containing sales data to forecast future sales volumes.

In [2]:
# Install necessary packages
!pip install pandas gluonts[mxnet] orjson

zsh:1: no matches found: gluonts[mxnet]


In [2]:
# Required Libraries
import json
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import SimpleFeedForwardEstimator, Trainer
from gluonts.evaluation import make_evaluation_predictions
from gluonts.evaluation import Evaluator

## Dataset Preparation

First, we load and prepare our dataset, splitting it into training and testing datasets.

In [1]:
# Dataset paths and column definitions
time_series_id_col = 'product_code'
date_col = 'sales_date'
target_col = 'volume'
freq = "1D"
prediction_length = 28
data_path = "data/isazi_ts_dataset.csv"

# Load the dataset
df = pd.read_csv(data_path, parse_dates=[date_col])
df.head()

NameError: name 'pd' is not defined

In [4]:
def split_time_series(df, prediction_length):
    """
    Function to split off a df of time series into time series, where the second time series
    includes the last `prediction_length` time steps. 
    """
    # Create an empty dataframe for train and validation sets
    train_df = pd.DataFrame()
    validation_df = pd.DataFrame()
    
    # Group by the time series identifier
    grouped = df.groupby(time_series_id_col)
    
    # Iterate over each group (i.e., each individual time series)
    for item_id, group in grouped:
        # Sort the group by date if it's not already sorted
        group = group.sort_index()
        
        # Define the split point
        split_point = len(group) - prediction_length
        
        # Split the data into training and validation sets
        train_group = group.iloc[:split_point]
        validation_group = group
        
        # Append to the respective dataframes
        train_df = pd.concat([train_df, train_group])
        validation_df = pd.concat([validation_df, validation_group])
    
    return train_df, validation_df
    
train_df, test_df = split_time_series(df, prediction_length)

## Model Training

We will train a simple feedforward neural network to predict future sales.

In [5]:
def train_predictor(df):
    """
    A very basic feed forward predictor using GluonTS with a training and validation set.
    """
    df = df.copy()
    df.set_index(date_col, inplace=True) # GluonTS wants the timestamp to be the index

    # Split the training data into training and validation sets
    train_df, val_df = split_time_series(df, prediction_length)
    # Create the Pandas datasets
    train_ds = PandasDataset.from_long_dataframe(train_df,
                                                target=target_col,
                                                item_id=time_series_id_col,
                                                freq=freq)

    val_ds = PandasDataset.from_long_dataframe(val_df,
                                                target=target_col,
                                                item_id=time_series_id_col,
                                                freq=freq)

    # Train a feed forward estimator
    estimator = SimpleFeedForwardEstimator(
        num_hidden_dimensions=[10],
        prediction_length=prediction_length,
        context_length=100,
        trainer=Trainer(ctx="cpu", epochs=10, learning_rate=1e-3, num_batches_per_epoch=100),
    )
    predictor = estimator.train(training_data=train_ds, validation_data=val_ds)
    return predictor
    
predictor = train_predictor(train_df)

100%|████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 13.96it/s, epoch=1/10, avg_epoch_loss=5.84]
13it [00:00, 15.68it/s, epoch=1/10, validation_avg_epoch_loss=5.37]
100%|████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 14.52it/s, epoch=2/10, avg_epoch_loss=5.38]
13it [00:00, 16.48it/s, epoch=2/10, validation_avg_epoch_loss=5.32]
100%|████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.81it/s, epoch=3/10, avg_epoch_loss=5.38]
13it [00:00, 17.87it/s, epoch=3/10, validation_avg_epoch_loss=5.23]
100%|█████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 14.93it/s, epoch=4/10, avg_epoch_loss=5.3]
13it [00:00, 16.24it/s, epoch=4/10, validation_avg_epoch_loss=5.26]
100%|█████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 14.61it/s, epoch=5/10, avg_epoch_loss=5.2]
13it [00:00, 16.32it/s, epoch=5/10, validation_avg_epoch_loss=5

## Making Predictions

Using the trained model to make predictions on the test dataset.

In [7]:
def make_predictions(predictor, df):
    """
    Make predictions with a GluonTS predictor and return them as a df.
    """
    df = df.copy()
    df.set_index(date_col, inplace=True) # GluonTS wants the timestamp to be the index
    dataset = PandasDataset.from_long_dataframe(df,
                                                target=target_col,
                                                item_id=time_series_id_col,
                                                freq=freq)
        
    forecast_it, _ = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=100,  # number of sample paths we want for evaluation
    )
    forecasts = list(forecast_it)

    # Initialize a list to hold all the records before converting to a DataFrame
    records = []
    for forecast in forecasts:
        item_id = forecast.item_id
        start_timestamp = forecast.start_date.start_time

        # GluonTS does probabilistic forecasting with a number of samples
        # Calculate mean targets across all samples for each date position
        mean_targets = forecast.samples.mean(axis=0)
        for i, target in enumerate(mean_targets):
            # Calculate what the timestamp should be for each predicted target
            timestamp = start_timestamp + i * pd.to_timedelta(freq)
            # Store the prediction as a record
            records.append({date_col: timestamp, time_series_id_col: item_id, target_col: target})

    # Convert the predictions from a list of records into a DataFrame
    preds_df = pd.DataFrame(records)
    preds_df.set_index(date_col, inplace=True)
    return preds_df

output_preds_path = "my_predictions.csv"
preds_df = make_predictions(predictor, test_df)
preds_df.to_csv(output_preds_path)

## Error Calculation

We compute the error between the forecasts and the actual sales to evaluate our model's performance. For this we main use overall relative error bias, the metric for the hackathon should probably be a weighted combination like `(1 - rE - 0.5*rB)*100`.

In [11]:
def compute_error(actuals_path, preds_path):
    """
    Computes the relative error and relative bias from csv files of predictions and actual targets.
    """
    # Read predicted and actuals from their respective files if not already provided
    actuals_df = pd.read_csv(actuals_path, parse_dates=[date_col])
    preds_df = pd.read_csv(preds_path, parse_dates=[date_col])
    
    # Rename 'target' column to add suffix for 'preds' and 'actuals' respectively
    actuals_df.rename(columns={target_col: f'{target_col}_actuals'}, inplace=True)
    preds_df.rename(columns={target_col: f'{target_col}_preds'}, inplace=True)
    
    # Merge the two dataframes on the timestamp column and the time series identifier column
    df = pd.merge(actuals_df, preds_df, on=[date_col, time_series_id_col])    
    
    actual_var = target_col + '_actuals'
    pred_var = target_col +'_preds'
    measure_level = [time_series_id_col, date_col]

    # Drop all unecessary columns
    keep_vars = list(set(measure_level + [actual_var, pred_var]))
    df_filtered = df.dropna(subset=[actual_var])[keep_vars]
    df_filtered.rename(columns={actual_var: 'A', pred_var: 'P'}, inplace=True)

    # Group by measure_level and aggregate A and P
    grouped = df_filtered.groupby(measure_level, observed=False).agg(A=('A', 'sum'), P=('P', 'sum'))

    # Calculate the errors initially at measure_level (not the absolute sum yet)
    grouped['E'] = (grouped['A'] - grouped['P']).abs()

    # Aggregate all data to one row
    grouped = grouped.sum()

    # Calculate relative error (rE) and relative bias (rB)
    grouped['rE'] = grouped['E'] / grouped['A']
    grouped['rB'] = (grouped['P'] - grouped['A']) / grouped['A']
    return grouped

output_preds_path = "my_predictions.csv"
report = compute_error(data_path, output_preds_path)
print("Error Report:")
print(report)

forecast_acc = (1 - (report.rE)) * 100
forecast_bias = report.rB * 100
acc_bias = forecast_acc - 0.5 * forecast_bias
print(f"Forecast Accuracy is: {forecast_acc:.2f}%")
print(f"Forecast Bias is: {forecast_bias:.2f}%")
print(f"Bias-weighted Accuracy: {acc_bias:.2f}%")

Error Report:
A     1.199319e+06
P     1.340588e+06
E     5.480017e+05
rE    4.569274e-01
rB    1.177912e-01
dtype: float64
Forecast Accuracy is: 54.31%
Forecast Bias is: 11.78%
Bias-weighted Accuracy: 48.42%
